In [1]:
import numpy as np
import pandas as pd

In [2]:
def mycov(x):
    x = x - x.mean(axis=0)
    U, s, V = np.linalg.svd(x, full_matrices = 0)
    C = np.dot(np.dot(V.T,np.diag(s**2)),V)
    return C / (x.shape[0] - 1)

In [3]:
n=145186
train = pd.read_csv("train_all.csv")
test  = pd.read_csv("test_all.csv")
ids   = test.loc[:,'id']
test  = test.drop(['id'], axis=1)
train = train.drop(['id'], axis=1)
test  = np.matrix(test)

class1_data = train[train['class']==1]
class1_data = class1_data.drop(['class'], axis=1)
class1_data = np.matrix(class1_data)
mean1       = class1_data.mean(0)
#covariance matrix for class1
s1          = mycov(class1_data)

class2_data = train[train['class']==2]
class2_data = class2_data.drop(['class'], axis=1)
class2_data = np.matrix(class2_data)
mean2       = class2_data.mean(0)
#covariance matrix for class2
s2          = mycov(class2_data)
#reshape mean to vector
mean1 = mean1.reshape(3,1)
mean2 = mean2.reshape(3,1)

#inverse of covariance matrixs
sigma1_inv =np.linalg.inv(s1)
sigma2_inv =np.linalg.inv(s2)

p_class1 = 40687/n
p_class2 = 104499/n

tr = p_class2/p_class1
#determinant of inverse of covariance matrixs
absolute1 = np.linalg.det(s1)
absolute2 = np.linalg.det(s2)

ref from Dr.Waleed A. Yousef lecture notes 
$$ \underbrace{x^T(\Sigma_2^{-1} - \Sigma_1^{-1})x}_{\text{Quadratic Term}} -\underbrace{ 2x^T(\Sigma_2^{-1}\mu_2-\Sigma_1^{-1}\mu_1) }_{\text{Linear Term}}+ (\mu^T_2\Sigma_2^{-1}\mu_2-\mu_1^T\Sigma_1^{-1}\mu_1)
\underset{G_2}{ \overset{G_1}{\lessgtr} } 2th +\text{ln}\bigg(\frac{|\Sigma_1|}{|\Sigma_2|}\bigg)$$


In [ ]:
log = np.log((absolute1/absolute2))

classifier = 2*tr + log

Scal = np.dot(mean2.T, np.dot(sigma2_inv,mean2) ) - np.dot(mean1.T,np.dot(sigma1_inv,mean1))

y_hat=[]

for val in test:
    val = np.array(val)
    val = val.reshape(3,1)
    Q = np.dot(val.T,np.dot((sigma2_inv-sigma1_inv) , val))
    L = np.dot(val.T, (np.dot(sigma2_inv,mean2) - np.dot(sigma1_inv,mean1)))
    pre = Q - 2*L + Scal
    if pre > classifier :
        y_hat.append(1)
    else :
        y_hat.append(2)

In [ ]:
#write y_hat in csv file
y_hat = pd.DataFrame(y_hat)
y_hat=pd.concat([ids, y_hat],axis=1)
y_hat.to_csv('cla_test_y1.csv', encoding='utf-8')